## Covid data for the counties I care about

The Washington Post has convenient data by state. I care about Washington DC, where I live, and how certain other locations are doing. The state-level data is not fine-grained enough for me. MSA and county level data are available online, but overwelming and not easily filterable to what I want. I'm creating this tool to provide historic data at the county level. I will use plotly for interactive visualizations and serve the website via FastAPI or put into Streamlit. I'll use GitHub actions and Prefect to fetch the data and make sure everything runs okay. I'll use Great Expectations for data quality checking and PyTest to check my code. 

I may use DVC to version my data.

At some later date, I may make an app that allows other users to choose which counties they want to include.

Imports and config

In [20]:
import pandas as pd
import plotly.express as px

pd.options.display.max_rows = 100


Read in data

In [2]:
df_2022 = pd.read_csv("us-counties-2022.csv", index_col="date")
df_2022


,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
date,,,,,,,,,
2022-01-01,USA-72999,Unknown,Puerto Rico,0,328.14,NaN,0,0.00,NaN
2022-01-01,USA-72153,Yauco,Puerto Rico,0,66.50,196.40,0,0.00,0.00
2022-01-01,USA-72151,Yabucoa,Puerto Rico,0,63.13,196.30,0,0.00,0.00
2022-01-01,USA-72149,Villalba,Puerto Rico,0,47.50,221.18,0,0.00,0.00
2022-01-01,USA-72147,Vieques,Puerto Rico,0,7.63,91.16,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...
2022-01-28,USA-69100,Rota,Northern Mariana Islands,0,0.00,0.00,0,0.00,0.00
2022-01-28,USA-78999,Unknown,Virgin Islands,0,0.00,NaN,1,0.22,NaN
2022-01-28,USA-78030,St. Thomas,Virgin Islands,6,32.75,63.43,0,0.43,0.83


In [3]:
df_2022.info()


<class 'pandas.core.frame.DataFrame'>
Index: 91102 entries, 2022-01-01 to 2022-01-28
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   geoid                91102 non-null  object 
 1   county               91102 non-null  object 
 2   state                91102 non-null  object 
 3   cases                91102 non-null  int64  
 4   cases_avg            91102 non-null  float64
 5   cases_avg_per_100k   90197 non-null  float64
 6   deaths               91102 non-null  int64  
 7   deaths_avg           91102 non-null  float64
 8   deaths_avg_per_100k  90197 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 7.0+ MB


Finding counties that could be tricky to match spelling/format.

In [4]:
df_2022[df_2022["county"].str.startswith("Alexandria")].head(2)


,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
date,,,,,,,,,
2022-01-01,USA-51510,Alexandria city,Virginia,0,281.14,176.34,0,0.38,0.24
2022-01-02,USA-51510,Alexandria city,Virginia,0,281.14,176.34,0,0.38,0.24


In [5]:
df_2022[df_2022["state"].str.startswith("District")].head(2)


,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
date,,,,,,,,,
2022-01-01,USA-11001,District of Columbia,District of Columbia,0,2103.0,297.98,0,0.4,0.06
2022-01-02,USA-11001,District of Columbia,District of Columbia,0,2103.0,297.98,0,0.4,0.06


In [6]:
df_2022[df_2022["state"].str.contains("York")].head(2)


,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
date,,,,,,,,,
2022-01-01,USA-36998,New York City,New York,45341,34646.38,415.58,20,27.89,0.33
2022-01-01,USA-36123,Yates,New York,28,13.25,53.19,0,0.14,0.57


Filter to counties of interest

In [7]:
counties = [
    "District of Columbia",
    "Wood",
    "Putnam",
    "Montgomery",
    "Prince George's",
    "Arlington",
    "Alexandria city",
    "New York City",  # README at NYT mentions some NE are city, not county
    "Allegheny",
    "Cook",
    "Baltimore",
    "Franklin",
    "Clermont",
    "Somerset",
    "Philadelphia",
    "Denver",
    "Boulder",
    "San Francisco",
    "Los Angeles",
    "Pima",
    "Manatee",
]


In [8]:
cols = ["county", "state", "geoid", "cases_avg_per_100k"]

df_2022_smaller = df_2022.loc[df_2022["county"].isin(counties), cols]
df_2022_smaller


,county,state,geoid,cases_avg_per_100k
date,,,,
2022-01-01,Wood,Wisconsin,USA-55141,82.19
2022-01-01,Wood,West Virginia,USA-54107,55.42
2022-01-01,Putnam,West Virginia,USA-54079,73.14
2022-01-01,Franklin,Washington,USA-53021,32.26
2022-01-01,Montgomery,Virginia,USA-51121,46.25
...,...,...,...,...
2022-01-28,Montgomery,Arkansas,USA-05097,104.93
2022-01-28,Franklin,Arkansas,USA-05047,152.41
2022-01-28,Pima,Arizona,USA-04019,222.82


See each state/county once.

In [9]:
df_2022_smaller.drop_duplicates(subset=["county", "state"])


,county,state,geoid,cases_avg_per_100k
date,,,,
2022-01-01,Wood,Wisconsin,USA-55141,82.19
2022-01-01,Wood,West Virginia,USA-54107,55.42
2022-01-01,Putnam,West Virginia,USA-54079,73.14
2022-01-01,Franklin,Washington,USA-53021,32.26
2022-01-01,Montgomery,Virginia,USA-51121,46.25
2022-01-01,Franklin,Virginia,USA-51067,68.83
2022-01-01,Arlington,Virginia,USA-51013,190.42
2022-01-01,Alexandria city,Virginia,USA-51510,176.34
2022-01-01,Franklin,Vermont,USA-50011,96.87


Convert geoid to FIPS code for plotting

In [10]:
df_2022["fips"] = df_2022["geoid"].str[-5:]
df_2022


,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k,fips
date,,,,,,,,,,
2022-01-01,USA-72999,Unknown,Puerto Rico,0,328.14,NaN,0,0.00,NaN,72999
2022-01-01,USA-72153,Yauco,Puerto Rico,0,66.50,196.40,0,0.00,0.00,72153
2022-01-01,USA-72151,Yabucoa,Puerto Rico,0,63.13,196.30,0,0.00,0.00,72151
2022-01-01,USA-72149,Villalba,Puerto Rico,0,47.50,221.18,0,0.00,0.00,72149
2022-01-01,USA-72147,Vieques,Puerto Rico,0,7.63,91.16,0,0.00,0.00,72147
...,...,...,...,...,...,...,...,...,...,...
2022-01-28,USA-69100,Rota,Northern Mariana Islands,0,0.00,0.00,0,0.00,0.00,69100
2022-01-28,USA-78999,Unknown,Virgin Islands,0,0.00,NaN,1,0.22,NaN,78999
2022-01-28,USA-78030,St. Thomas,Virgin Islands,6,32.75,63.43,0,0.43,0.83,78030


In [11]:
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91102 entries, 2022-01-01 to 2022-01-28
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   geoid                91102 non-null  object 
 1   county               91102 non-null  object 
 2   state                91102 non-null  object 
 3   cases                91102 non-null  int64  
 4   cases_avg            91102 non-null  float64
 5   cases_avg_per_100k   90197 non-null  float64
 6   deaths               91102 non-null  int64  
 7   deaths_avg           91102 non-null  float64
 8   deaths_avg_per_100k  90197 non-null  float64
 9   fips                 91102 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 7.6+ MB


Filter to fips codes of counties I want. 

If ever make into an app, will change to have folks choose State and then County from drop downs.


In [42]:
fips_counties = [
    "11001",
    "24033",
    "24031",
    "17031",
    "39173",
    "39137",
    "39113",
    "39049",
    "51013",
    "42111",
    "42003",
    "39025",
    "08031",
    "08013",
    "04019",
    "24005",
    "06037",
    "06075",
    "36061",
    "12081",
]

cols = ["county", "state", "fips", "cases_avg_per_100k"]

df_2022_counties = df_2022.loc[df_2022["fips"].isin(fips_counties), cols]
df_2022_counties


,county,state,fips,cases_avg_per_100k
date,,,,
2022-01-01,Arlington,Virginia,51013,190.42
2022-01-01,Somerset,Pennsylvania,42111,76.83
2022-01-01,Allegheny,Pennsylvania,42003,132.63
2022-01-01,Wood,Ohio,39173,109.09
2022-01-01,Putnam,Ohio,39137,54.00
...,...,...,...,...
2022-01-28,District of Columbia,District of Columbia,11001,52.97
2022-01-28,Denver,Colorado,08031,128.22
2022-01-28,Boulder,Colorado,08013,129.72


In [43]:
px.line(
    df_2022_counties, x=df_2022_counties.index, y="cases_avg_per_100k", color="county"
)

Montgomery is kind of a mess

7-day rolling average of cases as of yesterday's data

In [64]:
px.line(
    df_2022_counties, x=df_2022_counties.index, y="cases_avg_per_100k", color="county"
)

TODO: Include older date.

Read historic data and concatenate DataFrames.

## Map

Most recent 7 day moving average.

Scatter geo. 

Future direction: could make an animation over time. Could do choropleth too. Could show the DataFrame for just the most recent day in an app, too.

In [45]:
most_recent_date = df_2022_counties.index.max()

In [46]:
df_newest = df_2022_counties[df_2022_counties.index == most_recent_date].sort_values(
    by="cases_avg_per_100k"
)
df_newest


,county,state,fips,cases_avg_per_100k
date,,,,
2022-01-28,Baltimore,Maryland,24005,37.85
2022-01-28,Prince George's,Maryland,24033,38.05
2022-01-28,Montgomery,Maryland,24031,52.37
2022-01-28,District of Columbia,District of Columbia,11001,52.97
2022-01-28,Arlington,Virginia,51013,84.14
2022-01-28,Putnam,Ohio,39137,87.33
2022-01-28,Cook,Illinois,17031,88.31
2022-01-28,Franklin,Ohio,39049,93.89
2022-01-28,Allegheny,Pennsylvania,42003,108.56


In [47]:
# Need a shape file or to  get lat lon for each of the counties
# harder to find then expected. Found a mapping here: https://simplemaps.com/data/us-counties.


In [48]:
df_mapping = pd.read_csv('uscounties.csv')
df_mapping

,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population
0,Los Angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3207,-118.2248,10081570
1,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5198275
2,Harris,Harris,Harris County,48201,TX,Texas,29.8577,-95.3936,4646630
3,Maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4328810
4,San Diego,San Diego,San Diego County,6073,CA,California,33.0341,-116.7353,3316073
...,...,...,...,...,...,...,...,...,...
3137,Arthur,Arthur,Arthur County,31005,NE,Nebraska,41.5689,-101.6958,427
3138,McPherson,McPherson,McPherson County,31117,NE,Nebraska,41.5682,-101.0605,395
3139,King,King,King County,48269,TX,Texas,33.6166,-100.2558,237
3140,Loving,Loving,Loving County,48301,TX,Texas,31.8493,-103.5799,98


Need to add a 0 in front for the mapping county fips.

In [50]:
df_mapping['county_fips_str'] = df_mapping['county_fips'].astype(str).str.zfill(5)

In [55]:
df_ll = pd.merge(left=df_newest, right=df_mapping, how='left', left_on='fips', right_on='county_fips_str')
df_ll


,county_x,state,fips,cases_avg_per_100k,county_y,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,county_fips_str
0,Baltimore,Maryland,24005,37.85,Baltimore,Baltimore,Baltimore County,24005,MD,Maryland,39.4627,-76.6393,828018,24005
1,Prince George's,Maryland,24033,38.05,Prince George's,Prince George's,Prince George's County,24033,MD,Maryland,38.8295,-76.8473,908670,24033
2,Montgomery,Maryland,24031,52.37,Montgomery,Montgomery,Montgomery County,24031,MD,Maryland,39.1363,-77.2042,1043530,24031
3,District of Columbia,District of Columbia,11001,52.97,District of Columbia,District of Columbia,District of Columbia,11001,DC,District of Columbia,38.9047,-77.0163,692683,11001
4,Arlington,Virginia,51013,84.14,Arlington,Arlington,Arlington County,51013,VA,Virginia,38.8786,-77.1011,233464,51013
5,Putnam,Ohio,39137,87.33,Putnam,Putnam,Putnam County,39137,OH,Ohio,41.0221,-84.1317,33911,39137
6,Cook,Illinois,17031,88.31,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5198275,17031
7,Franklin,Ohio,39049,93.89,Franklin,Franklin,Franklin County,39049,OH,Ohio,39.9695,-83.0093,1290360,39049
8,Allegheny,Pennsylvania,42003,108.56,Allegheny,Allegheny,Allegheny County,42003,PA,Pennsylvania,40.4688,-79.9812,1221744,42003
9,Wood,Ohio,39173,127.22,Wood,Wood,Wood County,39173,OH,Ohio,41.3617,-83.6230,130150,39173


In [60]:
px.scatter_geo(
    df_ll,
    lat="lat",
    lon="lng",
    size="cases_avg_per_100k",
    scope="usa",
    color="cases_avg_per_100k",
    color
)


In [53]:
# px.choropleth(df_newest,  locations='fips', color='cases_avg_per_100k', scope='usa'  )
# %pip install plotly-geo pyshp
# %pip install shapely


In [21]:
import plotly.figure_factory as ff

# ff.create_choropleth(fips=df_newest['fips'], values=df_newest['cases_avg_per_100k'])

ImportError: geopandas, pyshp and shapely must be installed for this figure factory.

Run the following commands to install the correct versions of the following modules:

```
$ pip install geopandas==0.3.0
$ pip install pyshp==1.2.10
$ pip install shapely==1.6.3
```
If you are using Windows, follow this post to properly install geopandas and dependencies:http://geoffboeing.com/2014/09/using-geopandas-windows/

If you are using Anaconda, do not use PIP to install the packages above. Instead use conda to install them:

```
$ conda install plotly
$ conda install geopandas
```

In [ ]:
# %conda install plotly -c conda-forge
# %conda install -c plotly plotly-geo

In [ ]:
# %conda install geopandas -c conda-forge 

Convert geoid to FIPS code for plotting

In [ ]:
df_2022['fips'] = df_2022['geoid'].str[-5:]
df_2022

,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k,fips
date,,,,,,,,,,
2022-01-01,USA-72999,Unknown,Puerto Rico,0,328.14,NaN,0,0.00,NaN,72999
2022-01-01,USA-72153,Yauco,Puerto Rico,0,66.50,196.40,0,0.00,0.00,72153
2022-01-01,USA-72151,Yabucoa,Puerto Rico,0,63.13,196.30,0,0.00,0.00,72151
2022-01-01,USA-72149,Villalba,Puerto Rico,0,47.50,221.18,0,0.00,0.00,72149
2022-01-01,USA-72147,Vieques,Puerto Rico,0,7.63,91.16,0,0.00,0.00,72147
...,...,...,...,...,...,...,...,...,...,...
2022-01-28,USA-69100,Rota,Northern Mariana Islands,0,0.00,0.00,0,0.00,0.00,69100
2022-01-28,USA-78999,Unknown,Virgin Islands,0,0.00,NaN,1,0.22,NaN,78999
2022-01-28,USA-78030,St. Thomas,Virgin Islands,6,32.75,63.43,0,0.43,0.83,78030
